In [1]:
import json
import math

In [2]:
qasm = '''

OPENQASM 2.0;
include "hqslib1.inc";
qreg q0[6];


qreg a0[2];


creg init0[2];
creg meas0[6];



h q0[0];
cx q0[0], a0[0];
cx q0[0], q0[1];
cx q0[0], q0[4];
cx q0[0], q0[5];
cx q0[0], a0[0];
measure a0[0] -> init0[0];
reset a0[0];

h q0[2];
cx q0[2], a0[1];
cx q0[2], q0[3];
cx q0[2], q0[4];
cx q0[2], q0[5];
cx q0[2], a0[1];
measure a0[1] -> init0[1];
reset a0[1];




measure q0 -> meas0;
'''

In [3]:
with open("RyLength0.json", "r") as file:
    jobs = json.load(file)

In [4]:
def checks(r,i):
    if (r['init0'][i] != '00'):
        return False
    meas0 = r['meas0'][i]
    if (int(meas0[5]) + int(meas0[4]) + int(meas0[3]) + int(meas0[2]))%2 == 1:
        return False
    if (int(meas0[3]) + int(meas0[2]) + int(meas0[1]) + int(meas0[0]))%2 == 1:
        return False
    return True


def success(r,i):
    meas0 = r['meas0'][i]
    return (int(meas0[5]) + int(meas0[3]) + int(meas0[1]))%2 == 0

In [5]:
#Note: the cy flips the Y stabilizer of the control...so after that we should expect a y stabilizer measurement of 0 in q0
#Note: ycheck1[0] should be 0
totals = 0
failures = 0
shots = 0
for j in jobs:
    r = j['results']
    for i in range(0,len(r['init0'])):
        shots += 1
        if (checks(r,i)):
            totals += 1
            if (not success(r,i)):
                failures += 1
print('failures:', failures, 'totals:', totals)
print('logical failure rate:', failures/totals)
print('acceptance rate:', totals/shots)
print('acceptance rate standard deviation:', math.sqrt((totals/shots)*(shots-totals))/shots)

failures: 1 totals: 29206
logical failure rate: 3.4239539820584814e-05
acceptance rate: 0.9735333333333334
acceptance rate standard deviation: 0.0009267538727951132


In [6]:
p = failures/totals
n = totals
wilson = (1/(1 + 1/n))*(1/(2*n))*(math.sqrt(4*n*p*(1-p) + 1))
wilson_mean = 1/(1+ (1/n))*(p + (1/(2*n)))
print('logical failure rate confidence interval: (',  wilson_mean - wilson, ',',  wilson_mean + wilson, ')',)

logical failure rate confidence interval: ( 1.3078416946029236e-05 , 8.963668559788148e-05 )
